# ADC Simulationn (Linearität)
Version: 12.2020

Mit dieser Simulation sollen grundsätzlich RunUp und RunDown Funktionen

# Library laden


In [1]:
import SimADC_V1 as adc
import pandas as pd
import plotly.graph_objects as px

import plotly.io as pio
pio.renderers.default = 'iframe'


## Simulation starten


## Diagramm erstellen
Vout am Integrator

Beipiele und Dokumention finden sie auf der folgenden Seite:
https://plotly.com/python/<br>
https://plotly.com/python/bar-charts/#basic-bar-chart-with-plotlygraphobjects

In [2]:
# Ausgabe 100 Werten

cpu = adc.CPU()
integrator = adc.Integrator()

# 0 == keine besondere Form.
# z.B. 16 = der Schalter wird nochmal 1/16 umgeschaltet.
cpu.WaveForm = 0

x_vals = []
y_vals= []


integrator.VrefN = -7.04580
integrator.VrefP = 7.04480
integrator.ChargingInjection = 0.000

integrator.R_Vin   = 50.0 # k
integrator.R_VrefN = 50.01   # k
integrator.R_VrefP = 50.000  # k
cpu.RunUpCycles = 6250   # Anzahl der Rampen ( PLC 50Hz = 120ms )
cpu.RunUpCnt    = 32  # 3,2 us   # 10 Mhz (100 * 0.1us) = 10us

integrator.C = 0.33 #nF

# Auflösung: 6250 * 32 * 0,1 = 2.000.000
# Bei Vin +/-10V = 20V / Aufslösung = 10 uV (theoretisch) 

cpu.Connect(integrator )
#cpu.ResidueADC = adc.ResidueADC(cpu, 16) # Reste ADC mit n bit erzeugen


# Kalibration Array mit RunDown-Phase
# Parameter: Vadc Vin gemessen ADC
# Parameter: Vtrue Vin real
calibration_points = [
    (-6.998630569886022, -7.0),  # -Vref
    (0.00021130173965206957, 0.0),            # 0 V
    (6.999017956408719,   7.0)    # +Vref
]

# Initialisiere die Kalibrierung
#cpu.Calibration = adc.ADCCalibration(calibration_points)


integrator.Vin = -5.0
for a in range(0,200+1):
    
    cpu.Reset()
    
    #cpu.AutoZero()
    cpu.RunUp()
    #cpu.RunUp_V1()
    cpu.RunDown()
    cpu.CalcVin() # Berechnet den Wert der CPU - Vin 
    

    
    delta = (integrator.Vin - cpu.Vin) 
    x_vals.append( integrator.Vin)
    y_vals.append( delta)
    

    
    integrator.Vin = integrator.Vin + 0.05
    
    
    
    
    

In [3]:

#print(x_vals)


# neues Diagramm erstellen mit Titel
fig = px.Figure()


# Linie anzeigen
fig.add_trace( px.Scatter( x=x_vals, y=y_vals , name="Vout" )  )

fig.update_layout(
   title_text='Abweichung der Messung',
    yaxis_title="Delta in [V]",
    xaxis_title="Vin in [V]",
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)



fig.show()


